## Importación de librerías

In [ ]:
%pip install skimpy

In [1]:
import gc
import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimpy import skim
from sklearn.model_selection import train_test_split
import json

## Configuración de variables globales

In [ ]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
BASE_DIR = os.getcwd() # Base directory for the project,
DATASET_DIR = os.path.join(BASE_DIR, "datasets") # Input directory,
NA_VALUES = ['N/a', 'na', 'Na', 'NA', 'NAN', 'Nan', 'NaN', np.nan] # Consistent NA values,
EXPECTED_DTYPES = None
CHUNK_SIZE = SETUP_JSON['chunk_size'] # Chunk size for reading CSV files,
SAMPLE_FRACTION = SETUP_JSON['sample_fraction'] # Fraction of data to sample,
LABELS = SETUP_JSON['labels'] # Labels for the dataset
final_dataframe = pd.DataFrame() # Initialize an empty DataFrame for final output   

In [3]:
pd.set_option("display.max_columns", 80)

## Limpieza previa

In [ ]:
def get_majority_dtype(series):
    dtype_counts = series.map(type).value_counts()
    if not dtype_counts.empty:
        return dtype_counts.idxmax()
    return None

def convert_value(value, target_type):
    try:
        return target_type(value)
    except (ValueError, TypeError):
        return np.nan

def clean_and_convert_by_majority_type_chunked(input_filepath, output_filepath, chunksize=10000):
    """
    Limpia un archivo CSV en bloques, intentando convertir valores al tipo de dato mayoritario
    en cada columna antes de eliminarlos, y guarda el resultado sobrescribiendo un archivo.

    Args:
        input_filepath (str): Ruta del archivo CSV original.
        output_filepath (str): Ruta del archivo donde guardar el CSV limpio.
        chunksize (int): Tamaño del bloque para leer el archivo en partes.
    """

    cleaned_chunks = []

    for chunk in pd.read_csv(input_filepath, chunksize=chunksize):
        for col in chunk.columns:
            majority_dtype = get_majority_dtype(chunk[col])
            if majority_dtype:
                chunk[col] = chunk[col].apply(lambda x: convert_value(x, majority_dtype))

        chunk_cleaned = chunk.dropna()
        cleaned_chunks.append(chunk_cleaned)

    df_final = pd.concat(cleaned_chunks, ignore_index=True)

    # Guardar el dataset limpio
    df_final.to_csv(output_filepath, index=False)

    print(f"Dataset limpio guardado y sobrescrito en: '{output_filepath}'")

BASE_DIR = os.getcwd()  # Or specify a fixed base path if needed
DATASET_DIR = os.path.join(BASE_DIR, "datasets")  # Input directory
csv_files = glob(os.path.join(DATASET_DIR, "*.csv"))  # Find all CSV files

for file in csv_files:
    clean_and_convert_by_majority_type_chunked(file, file)  # Procesar archivo en chunks y sobrescribirlo

## Agrupar por clase

In [12]:
import pandas as pd

def contar_paquetes_por_secuencia(df, columna_labels='Label'):
    """
    Cuenta el número de paquetes en cada secuencia consecutiva de etiquetas.

    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos de los paquetes.
        columna_labels (str): El nombre de la columna que contiene las etiquetas
                              que definen las secuencias.

    Returns:
        pd.DataFrame: Un DataFrame con las columnas 'Labels' (la etiqueta de la secuencia),
                      'count' (el número de paquetes en esa secuencia), y
                      'sequence_id' (un identificador único para cada secuencia).
                      Las secuencias están en el orden en que aparecen.
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("La entrada 'df' debe ser un DataFrame de pandas.")
    if columna_labels not in df.columns:
        raise ValueError(f"La columna '{columna_labels}' no se encuentra en el DataFrame.")
    if df.empty:
        return pd.DataFrame(columns=[columna_labels, 'count', 'sequence_id'])

    # Identificar dónde cambia la etiqueta para marcar el inicio de una nueva secuencia.
    # df[columna_labels].shift() desplaza los valores de la columna una posición hacia abajo.
    # La primera etiqueta siempre se considera el inicio de una nueva secuencia (manejo de NaN de shift()).
    inicio_nueva_secuencia = (df[columna_labels] != df[columna_labels].shift())

    # Asignar un ID único a cada secuencia usando cumsum() sobre la serie booleana.
    # Cada vez que 'inicio_nueva_secuencia' es True, el acumulado aumenta, creando un nuevo ID.
    id_secuencia = inicio_nueva_secuencia.cumsum()

    # Agrupar por el ID de secuencia y la etiqueta, luego contar el tamaño de cada grupo.
    # Esto nos da el conteo de paquetes para cada secuencia.
    conteo_secuencias = df.groupby([id_secuencia, df[columna_labels]], sort=False).size()
    conteo_secuencias = conteo_secuencias.reset_index(name='count')

    # Renombrar la columna del ID de secuencia para mayor claridad.
    conteo_secuencias.rename(columns={columna_labels: 'Labels', columna_labels + '_x': 'Labels_temp'}, inplace=True)
    # El nombre de la columna de id_secuencia es el nombre de la columna_labels original
    # después del groupby, así que lo renombramos a 'sequence_id'.
    conteo_secuencias.rename(columns={conteo_secuencias.columns[0]: 'sequence_id'}, inplace=True)


    return conteo_secuencias[['sequence_id', 'Labels', 'count']]

df = pd.read_csv('datasets\\02-14-2018.csv')
# Aplicar la función
resultados = contar_paquetes_por_secuencia(df, 'Label')

print("Conteo de paquetes por secuencia:")
print(resultados)

ValueError: cannot insert Label, already exists

In [ ]:
first_chunk_overall = True # Flag to control header writing in the output file

csv_files = glob(os.path.join(DATASET_DIR, "*.csv")) # Find all CSV files
file_count = len(csv_files)
print(f"Found {file_count} files to process.")

if not csv_files:
    print("No CSV files found. Exiting.")
else:
    # Iterate through each found CSV file
    for i, file_path_str in enumerate(csv_files):
        file_path = Path(file_path_str)
        print(f"\nProcessing file {i+1}/{file_count}: {file_path.name}...")

        file_df = pd.read_csv(file_path, sep=',', low_memory=False, na_values=NA_VALUES, dtype=EXPECTED_DTYPES)
        file_df[file_df['clase'] == 'B']
        total_rows = sum(1 for _ in open(file_path, 'r')) - 1  # Subtract 1 for the header row
        sample_size = max(1, int(total_rows * SAMPLE_FRACTION))  # Ensure at least one row is sampled
        sample_df = pd.read_csv(file_path, sep=',', low_memory=False, na_values=NA_VALUES, nrows=sample_size, dtype=EXPECTED_DTYPES)
        print(f"  Sample of {sample_size} rows taken from {file_path.name} (shape: {sample_df.shape})...")

    print(f"\nFinished processing all files.")

Searching for CSV files in: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets
Found 10 files to process.

Processing file 1/10: 02-14-2018.csv...
  Sample of 104857 rows taken from 02-14-2018.csv (shape: (104857, 80))...

Processing file 2/10: 02-15-2018.csv...
  Sample of 104857 rows taken from 02-15-2018.csv (shape: (104857, 80))...

Processing file 3/10: 02-16-2018.csv...
  Sample of 104857 rows taken from 02-16-2018.csv (shape: (104857, 80))...

Processing file 4/10: 02-20-2018.csv...


KeyboardInterrupt: 

In [ ]:
def process_csv_in_chunks(input_filepath):
    """
    Procesa un archivo CSV en chunks y realiza operaciones en cada chunk.

    Args:
        input_filepath (str): Ruta del archivo CSV a procesar.
        chunksize (int): Tamaño del chunk para leer el archivo en partes.
    """
    chunk_list = []  # Lista para almacenar los chunks procesados

    for chunk in pd.read_csv(input_filepath, chunksize=CHUNK_SIZE, na_values=NA_VALUES, dtype=EXPECTED_DTYPES):
        # Realiza operaciones en cada chunk aquí
        chunk_cleaned = chunk.dropna()  # Ejemplo: eliminar filas con valores NaN
        chunk_list.append(chunk_cleaned)

    # Combina todos los chunks procesados en un DataFrame final
    final_df = pd.concat(chunk_list, ignore_index=True)
    return final_df

In [ ]:
first_chunk_overall = True # Flag to control header writing in the output file

csv_files = glob(os.path.join(DATASET_DIR, "*.csv")) # Find all CSV files
file_count = len(csv_files)
label_groups = {}
print(f"Found {file_count} files to process.")

# Iterate through each found CSV file
for i, file_path_str in enumerate(csv_files):
    file_path = Path(file_path_str)
    print(f"\nProcessing file {i+1}/{file_count}: {file_path.name}...")

    df = process_csv_in_chunks(file_path)
    for label in LABELS:
        label_groups[label] = df[df['Label'] == label]
print(f"\nFinished processing all files.")

In [18]:
print(f"Dimensiones del dataframe final: {final_dataframe.shape}")
print(f"Diferentes etiquetas encontradas en el dataframe: {final_dataframe['Label'].unique()}") # Display unique labels in the final DataFrame

Dimensiones del dataframe final: (8829012, 80)
Diferentes etiquetas encontradas en el dataframe: ['Benign' 'FTP-BruteForce' 'SSH-Bruteforce' 'DoS attacks-GoldenEye'
 'DoS attacks-Slowloris' 'DoS attacks-SlowHTTPTest' 'DoS attacks-Hulk'
 'Label' 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC' 'Brute Force -Web'
 'Brute Force -XSS' 'SQL Injection' 'Infilteration' 'Bot']


## Análisis del dataframe

In [15]:
skim(final_dataframe[:100000]) # Display a summary of the first 100,000 rows of the DataFrame

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100000 │ │ int64       │ 54    │                                                          │
│ │ Number of columns │ 80     │ │ float64     │ 24    │                                                          │
│ └───────────────────┴────────┘ │ string      │ 2     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column       ┃ NA ┃ NA % ┃ mean    ┃ sd       ┃ p0      ┃ p25     ┃ p50     ┃ p75     ┃ p100      ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩  │
│ │ Dst Port     │  0 │    0 │   21.51 │    70.46 │       0 │      21 │      21 │      21 │      9984 │   ▇    │  │
│ │ Protocol     │  0 │    0 │   5.997 │   0.1695 │       0 │       6 │       6 │       6 │        17 │    ▇   │  │
│ │ Flow         │  0 │    0 │   72640 │  2837000 │       1 │       1 │       2 │       2 │ 112600000 │   ▇    │  │
│ │ Duration     │    │      │         │          │         │         │         │         │           │        │  │
│ │ Tot Fwd Pkts │  0 │    0 │   1.005 │     0.35 │       1 │       1 │       1 │       1 │        91 │   ▇    │  │
│ │ Tot Bwd Pkts │  0 │    0 │   1.002 │   0.3151 │       0 │       1 │       1 │       1 │        83 │   ▇    │  │
│ │ TotLen Fwd   │  0 │    0 │  0.3747 │    33.61 │       0 │       0 │       0 │       0 │      7680 │   ▇    │  │
│ │ Pkts         │    │      │         │          │         │         │         │         │           │        │  │
│ │ TotLen Bwd   │  0 │    0 │  0.3878 │    22.09 │       0 │       0 │       0 │       0 │      2302 │   ▇    │  │
│ │ Pkts         │    │      │         │          │         │         │         │         │           │        │  │
│ │ Fwd Pkt Len  │  0 │    0 │  0.1612 │    8.806 │       0 │       0 │       0 │       0 │      1168 │   ▇    │  │
│ │ Max          │    │      │         │          │         │         │         │         │           │        │  │
│ │ Fwd Pkt Len  │  0 │    0 │ 0.01564 │    2.131 │       0 │       0 │       0 │       0 │       300 │   ▇    │  │
│ │ Min          │    │      │         │          │         │         │         │         │           │        │  │
│ │ Fwd Pkt Len  │  0 │    0 │ 0.05212 │    3.948 │       0 │       0 │       0 │       0 │     697.6 │   ▇    │  │
│ │ Mean         │    │      │         │          │         │         │         │         │           │        │  │
│ │ Fwd Pkt Len  │  0 │    0 │ 0.05479 │    3.065 │       0 │       0 │       0 │       0 │     452.9 │   ▇    │  │
│ │ Std          │    │      │         │          │         │         │         │         │           │        │  │
│ │ Bwd Pkt Len  │  0 │    0 │  0.2671 │    12.76 │       0 │       0 │       0 │       0 │       976 │   ▇    │  │
│ │ Max          │    │      │         │          │         │         │         │         │           │        │  │
│ │ Bwd Pkt Len  │  0 │    0 │ 0.01645 │    2.326 │       0 │       0 │       0 │       0 │       329 │   ▇    │  │
│ │ Min          │    │      │         │          │     

## Limpieza del dataframe